드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


데이터셋 복사

In [2]:
!cp "/content/drive/MyDrive/OD_project/raw/차량상태인식 (1).zip" /content

In [82]:
!cp -r "/content/drive/MyDrive/OD_project/Capture" /content

In [97]:
!cp -r "/content/drive/MyDrive/OD_project/사고영상_new.zip" /content

데이터셋 압축 해제

In [3]:
!unzip -q "/content/차량상태인식 (1).zip" -d /content/my_dataset

In [99]:
!unzip -q "/content/사고영상_new.zip" -d /content/sago

csv 내용에 맞게 파일 나누기

In [4]:
import os
import glob
import cv2
import shutil
import pandas as pd


def prepare_yolo_dataset_by_csv(root_path, csv_path, output_path):
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    df = df[['env', 'split']].dropna()
    df = df[df['split'].isin(['train', 'val', 'test'])]

    split_map = dict(zip(df['env'], df['split']))
    print(f"✅ CSV 로드 완료: {len(split_map)}개의 환경(env) 설정 읽음.")

    image_paths = glob.glob(os.path.join(root_path, "**/img/*.png"), recursive=True)
    print(f"총 {len(image_paths)}개의 이미지를 찾았습니다.")

    if len(image_paths) == 0:
        print("❌ 이미지를 찾지 못했습니다. root_path 확인 필요")
        return

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

    processed_count = 0
    missing_env_count = 0

    for img_path in image_paths:
        folder_name = img_path.split(os.sep)[-3]
        file_name = os.path.basename(img_path)

        target_split = split_map.get(folder_name)
        if not target_split:
            missing_env_count += 1
            continue

        new_base = f"{folder_name}_{file_name}"
        new_label = new_base.replace(".png", ".txt")

        org_label_path = img_path.replace("img", "new_txt").replace(".png", ".txt")
        if not os.path.exists(org_label_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w, _ = img.shape

        yolo_labels = []
        with open(org_label_path) as f:
            for line in f:
                data = list(map(float, line.split()))
                if len(data) < 5:
                    continue

                x1, y1, x2, y2 = data[:4]
                class_id = int(data[4])

                xc = ((x1 + x2) / 2) / w
                yc = ((y1 + y2) / 2) / h
                nw = (x2 - x1) / w
                nh = (y2 - y1) / h

                yolo_labels.append(
                    f"{class_id} {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}"
                )

        if yolo_labels:
            shutil.copy(
                img_path,
                os.path.join(output_path, target_split, "images", new_base)
            )
            with open(
                os.path.join(output_path, target_split, "labels", new_label), "w"
            ) as f:
                f.write("\n".join(yolo_labels))

            processed_count += 1

    print("-" * 30)
    print("✨ 처리 완료!")
    print(f"- 성공적으로 변환된 이미지: {processed_count}장")
    print(f"- CSV에 정의되지 않아 제외된 폴더: {missing_env_count}개")
    print(f"- 결과 저장소: {os.path.abspath(output_path)}")


In [5]:
prepare_yolo_dataset_by_csv(
    root_path="/content/my_dataset",
    csv_path="/content/drive/MyDrive/OD_project/split_assignment.csv",
    output_path="/content/my_car_dataset"
)

✅ CSV 로드 완료: 165개의 환경(env) 설정 읽음.
총 42096개의 이미지를 찾았습니다.
------------------------------
✨ 처리 완료!
- 성공적으로 변환된 이미지: 33187장
- CSV에 정의되지 않아 제외된 폴더: 8909개
- 결과 저장소: /content/my_car_dataset


ultralytics 다운로드

In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00


ResNet + FPN 모델 정의

In [7]:
#2-2) 모델 정의 (ResNet50-FPN + ROIAlign + shared head + 2 heads)

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.faster_rcnn import TwoMLPHead
from torchvision.models.detection.transform import GeneralizedRCNNTransform

def build_backbone_resnet50_fpn():
    # torchvision 버전 호환 처리
    try:
        from torchvision.models import ResNet50_Weights
        return resnet_fpn_backbone("resnet50", weights=ResNet50_Weights.DEFAULT)
    except Exception:
        # 구버전에서는 pretrained 인자를 받을 수 있음
        return resnet_fpn_backbone("resnet50", pretrained=True)

class ROIClassifier(nn.Module):
    def __init__(
        self,
        num_location=5,
        num_action=4,
        pos_weight=None,                 # torch.Tensor[4]
        min_size=640, max_size=1024,      # transform resize
        roi_output_size=7,
        roi_sampling_ratio=2,
        representation_size=1024,
        image_mean=(0.485,0.456,0.406),
        image_std=(0.229,0.224,0.225),
    ):
        super().__init__()

        self.transform = GeneralizedRCNNTransform(
            min_size=min_size,
            max_size=max_size,
            image_mean=list(image_mean),
            image_std=list(image_std),
        )

        self.backbone = build_backbone_resnet50_fpn()
        out_channels = self.backbone.out_channels  # usually 256

        self.roi_pooler = MultiScaleRoIAlign(
            featmap_names=["0","1","2","3"],
            output_size=roi_output_size,
            sampling_ratio=roi_sampling_ratio,
        )

        self.box_head = TwoMLPHead(
            in_channels=out_channels * roi_output_size * roi_output_size,
            representation_size=representation_size,
        )

        self.location_head = nn.Linear(representation_size, num_location)
        self.action_head   = nn.Linear(representation_size, num_action)

        # action loss용 pos_weight 저장
        if pos_weight is not None:
            self.register_buffer("pos_weight", pos_weight.clone().detach())
        else:
            self.pos_weight = None

    def forward(self, images, targets=None):
        """
        images: list[Tensor(3,H,W)]
        targets: list[dict] with keys:
          boxes: FloatTensor[N,4] xyxy (원본 픽셀 좌표계)
          location: LongTensor[N]
          action: FloatTensor[N,4] (0/1)
        """
        if self.training and targets is None:
            raise ValueError("Training requires targets")

        image_list, targets = self.transform(images, targets)
        features = self.backbone(image_list.tensors)  # dict[str,Tensor]

        boxes = [t["boxes"] for t in targets]
        pooled = self.roi_pooler(features, boxes, image_list.image_sizes)  # [sumN, C, S, S]
        pooled = pooled.flatten(start_dim=1)                               # [sumN, C*S*S]
        rep = self.box_head(pooled)                                        # [sumN, repr]

        loc_logits = self.location_head(rep)  # [sumN, 5]
        act_logits = self.action_head(rep)    # [sumN, 4]

        if self.training:
            loc_t = torch.cat([t["location"] for t in targets], dim=0)  # [sumN]
            act_t = torch.cat([t["action"] for t in targets], dim=0)    # [sumN,4]

            loss_loc = F.cross_entropy(loc_logits, loc_t)

            if self.pos_weight is None:
                loss_act = F.binary_cross_entropy_with_logits(act_logits, act_t)
            else:
                loss_act = F.binary_cross_entropy_with_logits(
                    act_logits, act_t, pos_weight=self.pos_weight
                )

            return {"loss_location": loss_loc, "loss_action": loss_act}

        # inference
        return F.softmax(loc_logits, dim=1), torch.sigmoid(act_logits)


YOLO, ResNet pt 적용

In [100]:
import glob
import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image as PILImage
import torchvision.transforms.functional as TF

# --- 1. 설정 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
#dataset_root = '/content/my_car_dataset'
dataset_root = '/content/sago'
output_root = '/content/sago_SafeDrive_T4_HighPerf/test_sequences_combined'
os.makedirs(output_root, exist_ok=True)

# 성민님 모델 가중치 경로 ★
TEAMMATE_WEIGHTS_PATH = "/content/drive/MyDrive/OD_project/best_by_microf1_bestth2.pt"

# --- 2. 모델 로드 ---
# (1) YOLO 모델 로드
if 'model' in globals():
    yolo_model = model
    print("✅ YOLO 모델(메모리) 사용")
else:
    from ultralytics import YOLO
    yolo_path = '/content/drive/MyDrive/OD_project/yolo11n_param_best.pt'
    if os.path.exists(yolo_path):
        yolo_model = YOLO(yolo_path)
        print("✅ YOLO 모델 새로 로드 완료")
    else:
        raise FileNotFoundError("❌ YOLO 모델을 찾을 수 없습니다.")

# (2) ROI 모델(팀원 모델) 로드
print(f"📂 팀원 모델 로드 시도: {TEAMMATE_WEIGHTS_PATH}")
if not os.path.exists(TEAMMATE_WEIGHTS_PATH):
    raise FileNotFoundError("❌ 팀원 모델 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

try:
    # 모델 구조 생성 (ROIClassifier 클래스가 메모리에 있어야 함)
    roi_model = ROIClassifier(num_location=5, num_action=4).to(device)

    # 가중치 파일 로드
    ckpt = torch.load(TEAMMATE_WEIGHTS_PATH, map_location=device, weights_only=False)

    # state_dict 키 처리
    if isinstance(ckpt, dict) and "model_state" in ckpt:
        state_dict = ckpt["model_state"]
    else:
        state_dict = ckpt

    # 로드 (strict=False로 설정하여 pos_weight 등 불일치 무시)
    roi_model.load_state_dict(state_dict, strict=False)
    roi_model.eval()
    print("✅ 팀원 모델(ROI) 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 중 오류 발생: {e}")
    raise e

"""
# --- 3. 시퀀스 그룹화 ---
all_test_images = glob.glob(os.path.join(dataset_root, 'test', 'images', '*.png'))
sequences = {}

for img_path in all_test_images:
    filename = os.path.basename(img_path)
    try:
        # 파일명에서 시퀀스 이름 추출
        seq_name = filename.rsplit('_', 1)[0]
        if seq_name not in sequences:
            sequences[seq_name] = []
        sequences[seq_name].append(img_path)
    except IndexError:
        pass

print(f"📂 총 {len(sequences)}개의 시퀀스를 발견했습니다. 통합 분석을 시작합니다...")
"""
# --- 3. 시퀀스 그룹화 (수정 버전) ---

# 패턴 1: 사용자가 지정한 폴더 바로 아래 모든 png 찾기
# 패턴 2: 하위 폴더 어디든 있는 모든 png 찾기 (recursive=True)
search_pattern = os.path.join(dataset_root, '**', '*.png')
all_test_images = glob.glob(search_pattern, recursive=True)

sequences = {}

if not all_test_images:
    print(f"❌ 이미지를 하나도 찾지 못했습니다. 현재 경로를 확인하세요: {dataset_root}")
else:
    for img_path in all_test_images:
        filename = os.path.basename(img_path)

        # 파일명에 '_'가 없는 경우를 대비한 안전한 분리
        if '_' in filename:
            seq_name = filename.rsplit('_', 1)[0]
        else:
            # 언더바가 없으면 폴더 이름을 시퀀스 이름으로 사용
            seq_name = os.path.basename(os.path.dirname(img_path))

        if seq_name not in sequences:
            sequences[seq_name] = []
        sequences[seq_name].append(img_path)

    print(f"📂 총 {len(sequences)}개의 시퀀스를 발견했습니다: {list(sequences.keys())}")

# --- 4. 내부 파이프라인 함수 ---
@torch.no_grad()
def process_single_frame(image_path, yolo_model, roi_model, device):
    img_cv = cv2.imread(image_path)
    if img_cv is None: return None, []

    filename = os.path.basename(image_path)

    # YOLO Input
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    img_pil = PILImage.fromarray(img_rgb)

    # YOLO Inference
    yolo_results = yolo_model(img_rgb, verbose=False)
    boxes = yolo_results[0].boxes.xyxy.cpu()
    clss = yolo_results[0].boxes.cls.cpu().int()
    names = yolo_model.names

    if len(boxes) == 0: return img_cv, []

    # ROI Input
    img_tensor = TF.to_tensor(img_pil).to(device)
    target_dict = [{"boxes": boxes.to(device)}]

    # ROI Inference
    loc_probs, act_probs = roi_model([img_tensor], target_dict)
    loc_preds = loc_probs.argmax(dim=1).cpu().numpy()
    act_probs = act_probs.cpu().numpy()

    # Config
    LOC_NAMES = ["My", "Next", "Opposite", "RoadSide", "None"]
    ACT_NAMES = ["Brake", "Left", "Right", "Hazard"]

    frame_detections = []
    output_img = img_cv.copy()

    for i, box in enumerate(boxes.numpy()):
        x1, y1, x2, y2 = map(int, box)
        cls_id = int(clss[i])
        class_name = names[cls_id] if names and cls_id in names else str(cls_id)

        loc_idx = int(loc_preds[i])
        loc_text = LOC_NAMES[loc_idx] if loc_idx < len(LOC_NAMES) else str(loc_idx)

        # Actions
        act_vals = act_probs[i]
        is_active = (act_vals > 0.5).astype(int)
        actions_str = [ACT_NAMES[j] for j, active in enumerate(is_active) if active]
        act_text = ",".join(actions_str) if actions_str else "Normal"

        # Append Data
        frame_detections.append({
            "Filename": filename,
            "Class_Name": class_name,
            "Location_Name": loc_text,
            "Action_String": act_text,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "Class_ID": cls_id,
            "Location_ID": loc_idx,
            "Brake": is_active[0], "Left": is_active[1], "Right": is_active[2], "Hazard": is_active[3]
        })

        # Draw
        label = f"[{class_name}] {loc_text}|{act_text}"
        cv2.rectangle(output_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        (w_t, h_t), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(output_img, (x1, y1 - 20), (x1 + w_t, y1), (0, 255, 0), -1)
        cv2.putText(output_img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    return output_img, frame_detections

"""
# --- 5. 전체 시퀀스 실행 ---
for seq_name, img_list in tqdm(sequences.items(), desc="Total Progress"):
    sorted_images = sorted(img_list)
    if not sorted_images: continue

"""
"""
    # 저장 경로
    seq_save_dir = os.path.join(output_root, seq_name)
    os.makedirs(seq_save_dir, exist_ok=True)

    video_path = os.path.join(seq_save_dir, f"{seq_name}_combined.mp4")
    csv_path = os.path.join(seq_save_dir, f"{seq_name}_combined.csv")

    # Video Init
    sample = cv2.imread(sorted_images[0])
    h, w = sample.shape[:2]
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))
"""
"""
    # 저장 경로
    seq_save_dir = os.path.join(output_root, seq_name)
    os.makedirs(seq_save_dir, exist_ok=True)

    # ==========================================================
    # ✅ [수정 포인트] 파일 제목 지정하기
    # 원하는 이름을 문자열로 넣으세요. (예: "Final_Result")
    my_custom_name = "Capture_result"

    # 만약 시퀀스 이름을 유지하면서 뒤에 글자만 붙이고 싶다면:
    # video_path = os.path.join(seq_save_dir, f"{seq_name}_{my_custom_name}.mp4")

    video_path = os.path.join(seq_save_dir, f"{my_custom_name}.mp4")
    csv_path = os.path.join(seq_save_dir, f"{my_custom_name}.csv")
    # ==========================================================

    # Video Init
    sample = cv2.imread(sorted_images[0])
    h, w = sample.shape[:2]
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

    # ... 이하 기존 코드와 동일 ...

    seq_detections = []

    for img_path in sorted_images:
        res_img, dets = process_single_frame(img_path, yolo_model, roi_model, device)
        if res_img is not None:
            out.write(res_img)
            seq_detections.extend(dets)

    out.release()

    if seq_detections:
        df = pd.DataFrame(seq_detections)
        cols = ["Filename", "x1", "y1", "x2", "y2", "Class_ID", "Location_ID", "Brake", "Left", "Right", "Hazard", "Class_Name", "Location_Name", "Action_String"]
        exist_cols = [c for c in cols if c in df.columns]
        df[exist_cols].to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n✨ 모든 통합 분석 완료! 결과 경로: {output_root}")
"""

# --- 5. 전체 시퀀스 실행 ---
for seq_name, img_list in tqdm(sequences.items(), desc="Total Progress"):
    sorted_images = sorted(img_list)
    if not sorted_images:
        continue

    # ==========================================================
    # ✅ [파일명 정제] seq_name에 .mp4 등이 포함된 경우 제거
    # 예: "video.mp4" -> "video_mp4" 또는 "video"로 변경
    clean_seq_name = seq_name.replace('.', '_')

    # 저장 경로 설정 (폴더명도 깨끗하게 유지)
    seq_save_dir = os.path.join(output_root, clean_seq_name)
    os.makedirs(seq_save_dir, exist_ok=True)

    # ✅ [파일명 지정]
    file_title = f"{clean_seq_name}_analysis"

    video_path = os.path.join(seq_save_dir, f"{file_title}.mp4")
    csv_path = os.path.join(seq_save_dir, f"{file_title}.csv")
    # ==========================================================

    # Video Init
    sample = cv2.imread(sorted_images[0])
    if sample is None:
        continue

    h, w = sample.shape[:2]
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

    seq_detections = []

    for img_path in sorted_images:
        res_img, dets = process_single_frame(img_path, yolo_model, roi_model, device)
        if res_img is not None:
            out.write(res_img)
            seq_detections.extend(dets)

    out.release()

    if seq_detections:
        df = pd.DataFrame(seq_detections)
        cols = ["Filename", "x1", "y1", "x2", "y2", "Class_ID", "Location_ID",
                "Brake", "Left", "Right", "Hazard", "Class_Name",
                "Location_Name", "Action_String"]
        exist_cols = [c for c in cols if c in df.columns]
        df[exist_cols].to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n✨ 모든 통합 분석 완료! 결과 경로: {output_root}")

✅ YOLO 모델 새로 로드 완료
📂 팀원 모델 로드 시도: /content/drive/MyDrive/OD_project/best_by_microf1_bestth2.pt


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


✅ 팀원 모델(ROI) 로드 성공!
📂 총 11개의 시퀀스를 발견했습니다: ['bb_1_210701_vehicle_217_21650.mp4', 'bb_1_160612_vehicle_212_53256.mp4', 'bb_1_141024_vehicle_46_180.mp4', 'bb_1_211026_vehicle_144_301.mp4', 'bb_1_170304_vehicle_114_075.mp4', 'bb_1_150402_vehicle_142_073.mp4', 'bb_1_181121_vehicle_117_045.mp4', 'bb_1_200714_vehicle_200_28840.mp4', 'bb_1_210414_vehicle_244_50609.mp4', 'bb_1_210114_vehicle_253_50073.mp4', 'bb_1_210522_vehicle_315_061.mp4']


Total Progress: 100%|██████████| 11/11 [00:46<00:00,  4.20s/it]


✨ 모든 통합 분석 완료! 결과 경로: /content/sago_SafeDrive_T4_HighPerf/test_sequences_combined


이후 단계를 위한 리소스 다운로드

In [11]:
# --- Install ---
!pip install numpy filterpy scipy imageio tqdm
!git clone https://github.com/abewley/sort.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=1a9c590822c73eb60deea935f1be4c71b6b64f43a99200cb53f5c13506053aae
  Stored in directory: /root/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy
Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 208 (delta 45), reused 40 (delta 40), pack-reused 159 (from 1)
Receiving objects: 100% (208/208), 1.20 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (76/76), done.


sort 함수 (수정 전)

In [ ]:
%%writefile /content/sort_tracker.py
# =====================================================
# SORT: Simple Online and Realtime Tracking
# Source adapted from https://github.com/abewley/sort
# =====================================================

import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment


def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1]) +
        (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - wh
    )
    return o


class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array(
            [[1,0,0,0,1,0,0],
            [0,1,0,0,0,1,0],
            [0,0,1,0,0,0,1],
            [0,0,0,1,0,0,0],
            [0,0,0,0,1,0,0],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1]]
        )
        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0]
        ])
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01

        # Fix: only take first 4 elements (coords)
        self.kf.x[:4] = bbox[:4].reshape((4,1))
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        # Fix: only take first 4 elements (coords)
        self.kf.update(bbox[:4].reshape((4,1)))

    def predict(self):
        if (self.kf.x[6] + self.kf.x[2]) <= 0:
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.kf.x)
        return self.kf.x


class Sort:
    def __init__(self, max_age=5, min_hits=2, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0,4))):
        self.frame_count += 1

        trks = np.zeros((len(self.trackers), 4))
        to_del = []
        for t, trk in enumerate(self.trackers):
            pos = trk.predict()
            trks[t] = pos[:4].reshape((1,4))
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(
            dets, trks, self.iou_threshold
        )

        for m in matched:
            self.trackers[m[1]].update(dets[m[0]])

        for i in unmatched_dets:
            self.trackers.append(KalmanBoxTracker(dets[i]))

        ret = []
        for trk in self.trackers:
            if trk.time_since_update < 1 and (trk.hits >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((trk.kf.x[:4].reshape((1,4)), [[trk.id]]), axis=1))
        return np.concatenate(ret) if len(ret) > 0 else np.empty((0,5))


def associate_detections_to_trackers(detections, trackers, iou_threshold):
    if len(trackers) == 0:
        return np.empty((0,2), dtype=int), np.arange(len(detections)), np.empty((0), dtype=int)

    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d,t] = iou(det, trk)

    matched_indices = linear_sum_assignment(-iou_matrix)
    matched_indices = np.array(list(zip(*matched_indices)))

    # Fix for IndexError when matched_indices is empty
    if len(matched_indices) == 0:
        matched_indices = np.empty((0, 2), dtype=int)

    unmatched_detections = []
    for d in range(len(detections)):
        if d not in matched_indices[:,0]:
            unmatched_detections.append(d)

    unmatched_trackers = []
    for t in range(len(trackers)):
        if t not in matched_indices[:,1]:
            unmatched_trackers.append(t)

    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))

    if len(matches) == 0:
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


Overwriting /content/sort_tracker.py


수정한 sort 함수

In [54]:
%%writefile /content/sort_tracker.py
# =====================================================
# SORT: Simple Online and Realtime Tracking
# Source adapted from https://github.com/abewley/sort
# =====================================================

import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment


def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1]) +
        (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - wh
    )
    return o


class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        # 7개 상태 변수: [u, v, s, r, u', v', s']
        # u,v: 중심점, s: 영역(넓이), r: 가로세로비
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array(
            [[1,0,0,0,1,0,0],
             [0,1,0,0,0,1,0],
             [0,0,1,0,0,0,1],
             [0,0,0,1,0,0,0],
             [0,0,0,0,1,0,0],
             [0,0,0,0,0,1,0],
             [0,0,0,0,0,0,1]]
        )
        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0]
        ])

        # --- [개선 포인트: 노이즈 파라미터 최적화] ---
        # R: 관측 노이즈. YOLO 박스의 미세한 떨림을 더 많이 무시하도록 상향 조정 (10 -> 20~30)
        self.kf.R[2:, 2:] *= 20.0

        # P: 초기 오차 공분산. 초기 위치는 확실치 않으므로 크게 설정
        self.kf.P[4:, 4:] *= 1000.
        self.kf.P *= 10.

        # Q: 프로세스 노이즈.
        # 속도(u', v') 항의 변화를 억제하여 궤적을 더 직선적이고 부드럽게 만듦
        self.kf.Q[-1, -1] *= 0.001
        self.kf.Q[4:, 4:] *= 0.001
        # ------------------------------------------

        # bbox 정보를 상태 변수로 변환 (u, v, s, r)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        u = bbox[0] + w/2.
        v = bbox[1] + h/2.
        s = w * h
        r = w / float(h)
        self.kf.x[:4] = np.array([u, v, s, r]).reshape((4, 1))

        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1

        # 입력받은 [x1, y1, x2, y2]를 [u, v, s, r]로 변환하여 업데이트
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        u = bbox[0] + w/2.
        v = bbox[1] + h/2.
        s = w * h
        r = w / float(h)
        self.kf.update(np.array([u, v, s, r]).reshape((4, 1)))

    def predict(self):
        """
        Kalman filter의 다음 상태를 예측하고, 예측된 상태를 [x1, y1, x2, y2] 형태로 반환
        """
        if (self.kf.x[6] + self.kf.x[2]) <= 0:
            self.kf.x[6] *= 0.0 # Prevent negative width/height
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1

        # Get the predicted state and convert it to bbox format [x1, y1, x2, y2]
        # The `get_state` method already does this conversion
        predicted_bbox = self.get_state()
        self.history.append(predicted_bbox) # Store bbox, not raw kf.x
        return predicted_bbox

    def get_state(self):
        """
        [u, v, s, r] 상태를 [x1, y1, x2, y2] 형태로 변환하여 반환
        """
        x = self.kf.x
        # Ensure s (area) and r (aspect ratio) are positive before sqrt
        s_val = max(0.01, x[2][0]) # Ensure area is not zero or negative
        r_val = max(0.01, x[3][0]) # Ensure aspect ratio is not zero or negative

        w = np.sqrt(s_val * r_val)
        h = s_val / w

        # Calculate x1, y1, x2, y2 from center (u,v), width (w), height (h)
        return np.array([x[0][0] - w/2., x[1][0] - h/2., x[0][0] + w/2., x[1][0] + h/2.]).reshape((1, 4))


class Sort:
    def __init__(self, max_age=5, min_hits=2, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0,4))):
        self.frame_count += 1

        trks = np.zeros((len(self.trackers), 4)) # To store predicted bboxes from trackers
        to_del = []
        for t, trk in enumerate(self.trackers):
            pos = trk.predict() # This calls the new predict method
            trks[t] = pos.reshape((1,4)) # predict now returns the bbox directly
            if np.any(np.isnan(pos)) or pos[0,2] <= pos[0,0] or pos[0,3] <= pos[0,1]: # Check for invalid bboxes
                to_del.append(t)

        # Remove invalid trackers before association
        for t in reversed(to_del):
            self.trackers.pop(t)
        trks = np.delete(trks, to_del, axis=0)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(
            dets, trks, self.iou_threshold
        )

        for m in matched:
            self.trackers[m[1]].update(dets[m[0]])

        for i in unmatched_dets:
            self.trackers.append(KalmanBoxTracker(dets[i]))

        ret = []
        for trk_idx, trk in enumerate(self.trackers):
            if trk.time_since_update < 1 and (trk.hits >= self.min_hits or self.frame_count <= self.min_hits):
                # Use the current state from the tracker (which is in bbox format after update/predict)
                current_bbox = trk.get_state() # Use get_state to get the current bbox
                ret.append(np.concatenate((current_bbox, [[trk.id]]), axis=1))

            # Delete old trackers
            if trk.time_since_update > self.max_age:
                to_del.append(trk_idx)

        # Delete trackers that have exceeded max_age
        for t in reversed(to_del):
            if t < len(self.trackers):
                self.trackers.pop(t)

        return np.concatenate(ret) if len(ret) > 0 else np.empty((0,5))


def associate_detections_to_trackers(detections, trackers, iou_threshold):
    if len(trackers) == 0:
        return np.empty((0,2), dtype=int), np.arange(len(detections)), np.empty((0), dtype=int)

    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d,t] = iou(det, trk) # det and trk are now both [x1, y1, x2, y2]

    matched_indices = linear_sum_assignment(-iou_matrix)
    matched_indices = np.array(list(zip(*matched_indices)))

    # Fix for IndexError when matched_indices is empty
    if len(matched_indices) == 0:
        matched_indices = np.empty((0, 2), dtype=int)

    unmatched_detections = []
    for d in range(len(detections)):
        if d not in matched_indices[:,0]:
            unmatched_detections.append(d)

    unmatched_trackers = []
    for t in range(len(trackers)):
        if t not in matched_indices[:,1]:
            unmatched_trackers.append(t)

    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))

    if len(matches) == 0:
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


Overwriting /content/sort_tracker.py


만들어진 sort 함수 저장하기

In [55]:
import sys
sys.path.insert(0, "/content")

from sort_tracker import Sort

tracker = Sort()
print("SORT import OK")

SORT import OK


In [56]:
import cv2
import os
import glob
import numpy as np
import imageio
from collections import deque
from tqdm import tqdm

RiskManager(수정 전)

In [ ]:
class IntentAwareRiskManager:
    def __init__(self, window_size=5):
        self.window_size = window_size
        self.history = {}  # {track_id: deque[(cx, cy)]}

    def _update_history(self, track_id, center):
        if track_id not in self.history:
            self.history[track_id] = deque(maxlen=self.window_size)
        self.history[track_id].append(center)

    def _is_long_static(self, centers, move_thresh=3.0, min_static_frames=6):
        if len(centers) < min_static_frames + 1:
            return False

        diffs = [
            np.linalg.norm(np.array(centers[i]) - np.array(centers[i-1]))
            for i in range(1, len(centers))
        ]
        return all(d < move_thresh for d in diffs[-min_static_frames:])

    def _compute_2d_traj_metrics(self, centers, fps, img_w, img_h, eps=1e-3):
        """
        return: ttc, v_rel, d_curr
        """
        if len(centers) < 2:
            return float('inf'), 0.0, float('inf')

        ego = np.array([img_w / 2, img_h])
        p_prev = np.array(centers[-2])
        p_curr = np.array(centers[-1])

        d_prev = np.linalg.norm(p_prev - ego)
        d_curr = np.linalg.norm(p_curr - ego)
        v_rel = (d_prev - d_curr) * fps  # px/sec

        # 접근 안 하면 무효
        if v_rel < 5.0:
            return float('inf'), v_rel, d_curr

        # 장기 정지 필터
        if self._is_long_static(centers):
            return float('inf'), v_rel, d_curr

        ttc = d_curr / max(v_rel, eps)
        return ttc, v_rel, d_curr

    def calculate_risk(self, track_id, bbox, location, actions, fps, img_w, img_h):
        cx = (bbox[0] + bbox[2]) / 2
        cy = (bbox[1] + bbox[3]) / 2
        self._update_history(track_id, (cx, cy))
        centers = self.history[track_id]

        # --- Step 1: Physical risk ---
        ttc, v_rel, d_curr = self._compute_2d_traj_metrics(
            centers, fps, img_w, img_h
        )

        base_score = 0.0
        if ttc < 5.0:
            base_score = min(50.0, 50.0 * (3.0 / max(ttc, 0.5)))

        # --- Step 2: Semantic reasoning ---
        semantic_bonus = 0.0
        omega = 1.0
        reason = "Normal"

        # Ignore irrelevant zones
        if location in [2, 3, 4]:
            return {
                "score": 0.0,
                "ttc": "Safe",
                "reason": "Ignored",
                "level": "SAFE"
            }

        HIGH_SPEED = 80.0  # px/sec

        # Ego lane
        if location == 0:
            if ttc < 2.0 and v_rel > HIGH_SPEED:
                reason = "Rapid Approach"
                semantic_bonus = 10.0
            elif ttc < 2.0:
                reason = "Close Following"
            else:
                reason = "Normal Deceleration"

        # Adjacent lane (cut-in)
        elif location == 1:
            if actions[1] == 1 or actions[2] == 1:
                reason = "Dangerous Cut-in"
                semantic_bonus = 10.0
                omega = 1.3
            else:
                reason = "Approaching"
                omega = 0.8

        final_risk = min(base_score * omega + semantic_bonus, 100.0)

        return {
            "score": round(final_risk, 2),
            "ttc": round(ttc, 2) if ttc != float('inf') else "Safe",
            "reason": reason,
            "level": self._get_risk_level(final_risk),
        }

    def _get_risk_level(self, score):
        if score > 50: return "DANGER"
        if score > 20: return "WARNING"
        return "SAFE"


수정한 버전 RiskManager

In [85]:
import numpy as np
from collections import deque

class IntentAwareRiskManager:
    def __init__(self, window_size=10):
        self.window_size = window_size
        self.history = {} # {tid: deque([d1, d2, ...])}

    def calculate_risk(self, track_id, bbox, location, actions, fps, img_w, img_h):
        # 1. 자차 위치 정의 (p_ego)
        ego_p = np.array([img_w / 2, img_h])

        # 2. 객체 중심점 추출 (c_x, c_y)
        curr_p = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])

        # 3. 거리 계산 (원근 왜곡 보정을 위한 y 가중치 고려 권장)
        # 여기서는 논문 수식대로 유클리드 거리 d_t 계산
        d_t = np.linalg.norm(curr_p - ego_p)

        if track_id not in self.history:
            self.history[track_id] = deque(maxlen=self.window_size)
        self.history[track_id].append(d_t)

        # --- Step 1: v_rel 및 TTC 계산 (전략 2 적용) ---
        d_history = list(self.history[track_id])
        if len(d_history) < 3:
            return self._safe_response()

        # 단순 차이 대신 선형 회귀로 기울기(v_rel) 추출
        x = np.arange(len(d_history))
        slope, _ = np.polyfit(x, d_history, 1)
        v_rel = -slope * fps  # 거리가 줄어들면 slope는 음수이므로 -를 붙여 접근 속도로 변환

        # 접근 속도 임계값 필터링 (v_rel < threshold)
        if v_rel < 5.0:
            return self._safe_response()

        # TTC 계산
        ttc = d_t / max(v_rel, 1e-3)

        # --- Step 2: Base Risk Score (수식 적용) ---
        base_score = 0.0
        if ttc < 5.0:
            # Score_base = min(50, 50 * (3 / max(TTC, 0.5)))
            base_score = min(50.0, 50.0 * (3.0 / max(ttc, 0.5)))

        # --- Step 3: Semantic Risk Amplification ---
        if location in [2, 3, 4]: # 도로 문맥 필터링
            return self._safe_response("Ignored Zone")

        omega = 1.0
        bonus = 0.0
        reason = "Normal"

        if location == 0: # 주행 차로 (Ego Lane)
            if ttc < 2.0 and v_rel > 80.0: # 임계값은 픽셀 속도 기준 튜닝 필요
                reason = "Rapid Approach"
                bonus = 15.0
                omega = 1.2
            elif ttc < 2.0:
                reason = "Close Following"
                bonus = 5.0
            else:
                reason = "Normal Deceleration"

        elif location == 1: # 인접 차로 (Adjacent)
            if actions[1] == 1 or actions[2] == 1: # 방향 지시등 활성화
                reason = "Dangerous Cut-in"
                omega = 1.5
                bonus = 20.0
            else:
                reason = "Approaching"
                omega = 0.8

        # --- 최종 점수 산출 ---
        final_score = min(base_score * omega + bonus, 100.0)

        return {
            "score": round(final_score, 2),
            "ttc": round(ttc, 2),
            "reason": reason,
            "level": self._get_risk_level(final_score)
        }

    def _get_risk_level(self, score):
        if score >= 50: return "DANGER"
        elif score >= 25: return "WARNING"
        #elif score >= 30: return "CAUTION"
        else: return "SAFE"

    def _safe_response(self, reason="Safe"):
        return {"score": 0.0, "ttc": "Safe", "reason": reason, "level": "SAFE"}

(XXX 아래거 실행!!) sort, RiskManager 통합 실행 수정버전 (영상으로 저장) (특정 파일만 저장)

In [92]:
import pandas as pd
import cv2
import os
import numpy as np
from tqdm import tqdm

# Sort 트래커에서 iou 함수를 가져옵니다.
import importlib
import sort_tracker
importlib.reload(sort_tracker) # Reload the module to ensure latest changes are applied
from sort_tracker import Sort, iou

# 1. 먼저 CSV 파일을 로드합니다. (경로는 본인의 환경에 맞게 수정)
DET_CSV_PATH = "/content/Capture_SafeDrive_T4_HighPerf/test_sequences_combined/bb_1_140613_vehicle_224_55460_mp4_20260114/bb_1_140613_vehicle_224_55460_mp4_20260114_analysis.csv"
all_det_df = pd.read_csv(DET_CSV_PATH)

# OUTPUT_ROOT_RISK 변수 정의 추가
OUTPUT_ROOT_RISK = "/content/Capture_SafeDrive_T4_HighPerf/risk_video"

# RISK_COLORS 정의 추가 (이전 셀에서 정의되었으나 명확성을 위해 추가)
RISK_COLORS = {
    "SAFE": (0, 255, 0),       # Green
    "WARNING": (0, 165, 255),  # Orange
    "DANGER": (0, 0, 255),
    "CAUTION": (0, 255, 255) # Yellow
}

"""
def process_sequence_from_csv(seq_name, image_paths, output_dir, det_df):
    os.makedirs(output_dir, exist_ok=True)
    video_path = os.path.join(output_dir, f"{seq_name}_risk_analysis.mp4")

    image_paths = sorted(image_paths)
    if not image_paths: return

    # Init Tracker & Risk Manager
    tracker = Sort(max_age=5, min_hits=1, iou_threshold=0.3)
    risk_manager = IntentAwareRiskManager(window_size=10)

    # Video Writer Init
    sample_img = cv2.imread(image_paths[0])
    h, w = sample_img.shape[:2]
    fps = 10.0
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    print(f"🎬 Processing {seq_name} from CSV...")

    # 해당 시퀀스 데이터만 필터링
    # NOTE: 'frame' 컬럼이 없으므로 'Filename'을 기반으로 필터링하도록 수정
    seq_data = det_df[det_df['Filename'].apply(lambda x: x.startswith(seq_name))].copy()
    seq_data['frame'] = seq_data['Filename'].apply(lambda x: int(os.path.basename(x).split('_')[-1].replace('.png', '')))

    for frame_idx, img_path in enumerate(tqdm(image_paths, leave=False)):
        img_cv = cv2.imread(img_path)
        if img_cv is None: continue

        # --- [변경 파트 1: 모델 호출 대신 CSV 필터링] ---
        # 현재 프레임에 해당하는 검출 데이터 추출
        frame_filename = os.path.basename(img_path)
        current_frame_dets = seq_data[seq_data['Filename'] == frame_filename].copy()

        # SORT 입력을 위해 [x1, y1, x2, y2, conf] 형태로 변환 (conf는 CSV에 없으므로 임시로 1.0 사용)
        # CSV에 conf가 없는 경우를 대비해 1.0으로 임시 설정. 실제 conf 값이 있다면 해당 컬럼 사용
        if 'conf' not in current_frame_dets.columns:
            current_frame_dets['conf'] = 1.0
        dets_to_sort = current_frame_dets[['x1', 'y1', 'x2', 'y2', 'conf']].values

        # 2. SORT Tracking (기존과 동일)
        track_dets = tracker.update(dets_to_sort)

        # 3. 위험도 계산 (기존 ROI 모델 호출부 삭제)
        if len(track_dets) > 0:
            for trk in track_dets:
                x1, y1, x2, y2, track_id = trk

                # --- [변경 파트 2: CSV에서 의미 정보(loc, action) 매칭] ---
                # 트래킹된 박스와 가장 가까운 원본 CSV 박스 찾기
                if len(current_frame_dets) > 0:
                    # YOLO의 박스와 SORT의 트랙킹된 박스 간의 IoU를 계산하여 매칭
                    matched_iou_val = -1
                    matched_row = None
                    for _, det_row in current_frame_dets.iterrows():
                        iou_val = iou(trk[:4], det_row[['x1', 'y1', 'x2', 'y2']].values)
                        if iou_val > matched_iou_val and iou_val > 0.1: # IoU 임계값 설정
                            matched_iou_val = iou_val
                            matched_row = det_row

                    if matched_row is not None:
                        loc_idx = int(matched_row['Location_ID'])
                        # CSV 컬럼명에 따라 act_brake, act_left 등으로 구성된 값을 배열로 만듦
                        is_active = matched_row[['Brake', 'Left', 'Right', 'Hazard']].values.astype(int)

                        # 4. Risk Calculation (기존과 동일)
                        risk_result = risk_manager.calculate_risk(
                            track_id=int(track_id),
                            bbox=[x1, y1, x2, y2],
                            location=loc_idx,
                            actions=is_active,
                            fps=fps, img_w=w, img_h=h
                        )

                        # 5. 시각화 (기존과 동일)
                        level = risk_result["level"]
                        color = RISK_COLORS.get(level, (0, 255, 0))
                        cv2.rectangle(img_cv, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                        loc_names = ["My", "Next", "Opposite", "RoadSide", "None"]
                        info_text = f"ID:{int(track_id)} | {loc_names[loc_idx] if loc_idx < len(loc_names) else '??'}"
                        cv2.putText(img_cv, info_text, (int(x1), int(y1) - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        cv2.putText(img_cv, f"{level} ({risk_result['score']})", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                        if level != "SAFE":
                            cv2.putText(img_cv, risk_result["reason"], (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(img_cv)

    out.release()

"""
# --- 실행 부분 ---
for seq_name, img_list in sequences.items():
    process_sequence_from_csv(
        seq_name,
        img_list,
        OUTPUT_ROOT_RISK,
        all_det_df  # 모델 대신 데이터프레임 전달
    )
    """

# --- 실행 부분 ---

# 1. CSV 파일 안에 어떤 시퀀스 이름들이 있는지 리스트로 추출합니다.
# Filename 컬럼의 값(예: 'CounterClock2_Fog_001.png')에서 시퀀스 이름만 뽑아냅니다.
csv_sequence_names = all_det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0]).unique()

print(f"✅ CSV에 포함된 시퀀스 목록: {csv_sequence_names}")

for seq_name, img_list in sequences.items():
    # 2. 현재 처리하려는 폴더명(seq_name)이 CSV 리스트에 들어있는지 확인합니다.
    if seq_name in csv_sequence_names:
        process_sequence_from_csv(
            seq_name,
            img_list,
            OUTPUT_ROOT_RISK,
            all_det_df
        )
    else:
        # CSV에 정보가 없는 폴더는 그냥 건너뜁니다.
        print(f"⏩ Skipping {seq_name}: No matching data found in CSV.")
        """
def process_sequence_from_csv(seq_name, image_paths, output_dir, det_df):
    # 마침표 문제 방지를 위해 안전한 시퀀스명 사용
    safe_seq_name = seq_name.replace('.', '_')
    os.makedirs(output_dir, exist_ok=True)
    video_path = os.path.join(output_dir, f"{safe_seq_name}_risk_analysis.mp4")

    image_paths = sorted(image_paths)
    if not image_paths: return

    # Init Tracker & Risk Manager
    tracker = Sort(max_age=5, min_hits=1, iou_threshold=0.3)
    risk_manager = IntentAwareRiskManager(window_size=10)

    # Video Writer Init
    sample_img = cv2.imread(image_paths[0])
    h, w = sample_img.shape[:2]
    fps = 10.0
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    print(f"🎬 Processing {safe_seq_name} from CSV...")

    # --- [수정 포인트] 시퀀스 데이터 필터링 ---
    # Filename에서 시퀀스 부분만 추출하여 seq_name과 일치하는지 확인
    seq_data = det_df[det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0] == seq_name)].copy()

    if seq_data.empty:
        print(f"⚠️ Warning: No data found for {seq_name} in CSV.")
        return

    for frame_idx, img_path in enumerate(tqdm(image_paths, leave=False)):
        img_cv = cv2.imread(img_path)
        if img_cv is None: continue

        frame_filename = os.path.basename(img_path)
        current_frame_dets = seq_data[seq_data['Filename'] == frame_filename].copy()

        if 'conf' not in current_frame_dets.columns:
            current_frame_dets['conf'] = 1.0
        dets_to_sort = current_frame_dets[['x1', 'y1', 'x2', 'y2', 'conf']].values

        track_dets = tracker.update(dets_to_sort)

        if len(track_dets) > 0:
            for trk in track_dets:
                x1, y1, x2, y2, track_id = trk
                matched_iou_val = -1
                matched_row = None
                for _, det_row in current_frame_dets.iterrows():
                    iou_val = iou(trk[:4], det_row[['x1', 'y1', 'x2', 'y2']].values)
                    if iou_val > matched_iou_val and iou_val > 0.1:
                        matched_iou_val = iou_val
                        matched_row = det_row

                if matched_row is not None:
                    loc_idx = int(matched_row['Location_ID'])
                    is_active = matched_row[['Brake', 'Left', 'Right', 'Hazard']].values.astype(int)

                    risk_result = risk_manager.calculate_risk(
                        track_id=int(track_id),
                        bbox=[x1, y1, x2, y2],
                        location=loc_idx,
                        actions=is_active,
                        fps=fps, img_w=w, img_h=h
                    )

                    level = risk_result["level"]
                    color = RISK_COLORS.get(level, (0, 255, 0))
                    cv2.rectangle(img_cv, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                    loc_names = ["My", "Next", "Opposite", "RoadSide", "None"]
                    info_text = f"ID:{int(track_id)} | {loc_names[loc_idx] if loc_idx < 5 else '??'}"
                    cv2.putText(img_cv, info_text, (int(x1), int(y1) - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.putText(img_cv, f"{level} ({risk_result['score']})", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    if level != "SAFE":
                        cv2.putText(img_cv, risk_result["reason"], (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(img_cv)
    out.release()

# --- 실행 부분 ---
# 1. CSV 내의 시퀀스 이름 리스트 추출 (RSPLIT 기준)
csv_sequence_names = all_det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0]).unique()

print(f"✅ CSV에 포함된 시퀀스 목록: {csv_sequence_names}")

for seq_name, img_list in sequences.items():
    # sequences의 key(seq_name)에 마침표가 있을 수 있으므로 변환하여 비교
    processed_seq_name = seq_name.replace('.', '_')

    if processed_seq_name in csv_sequence_names:
        process_sequence_from_csv(
            processed_seq_name, # 변환된 이름을 전달
            img_list,
            OUTPUT_ROOT_RISK,
            all_det_df
        )
    else:
        print(f"⏩ Skipping {processed_seq_name}: No matching data found in CSV.")


🎬 Processing bb_1_140613_vehicle_224_55460.mp4_20260114 from CSV...


ValueError: invalid literal for int() with base 10: '143204.270'

In [111]:
import pandas as pd
import cv2
import os
import numpy as np
from tqdm import tqdm
import importlib

# Sort 트래커에서 iou 함수 로드
import sort_tracker
importlib.reload(sort_tracker)
from sort_tracker import Sort, iou

# 1. 경로 설정
# [수정 포인트] 이전 단계에서 생성된 CSV 파일의 정확한 경로를 입력하세요.
DET_CSV_PATH = "/content/sago_SafeDrive_T4_HighPerf/test_sequences_combined/bb_1_211026_vehicle_144_301_mp4/bb_1_211026_vehicle_144_301_mp4_analysis.csv"
OUTPUT_ROOT_RISK = "/content/sago_SafeDrive_T4_HighPerf/risk_video"

all_det_df = pd.read_csv(DET_CSV_PATH)

# 위험도 색상 정의
RISK_COLORS = {
    "SAFE": (0, 255, 0),       # Green
    "WARNING": (0, 165, 255),  # Orange
    "DANGER": (0, 0, 255),     # Red
    "CAUTION": (0, 255, 255)   # Yellow
}

# 2. 핵심 처리 함수 정의
def process_sequence_from_csv(seq_name, image_paths, output_dir, det_df):
    # 마침표 문제 방지를 위해 안전한 시퀀스명 사용
    safe_seq_name = seq_name.replace('.', '_')
    os.makedirs(output_dir, exist_ok=True)
    video_path = os.path.join(output_dir, f"{safe_seq_name}_risk_analysis.mp4")

    image_paths = sorted(image_paths)
    if not image_paths: return

    # Init Tracker & Risk Manager (IntentAwareRiskManager가 메모리에 있어야 함)
    tracker = Sort(max_age=5, min_hits=1, iou_threshold=0.3)
    risk_manager = IntentAwareRiskManager(window_size=10)

    # Video Writer Init
    sample_img = cv2.imread(image_paths[0])
    h, w = sample_img.shape[:2]
    fps = 10.0
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    print(f"🎬 Processing {safe_seq_name} from CSV...")

    # 시퀀스 데이터 필터링 (파일명 규칙에 맞춤)
    seq_data = det_df[det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0] == seq_name)].copy()

    if seq_data.empty:
        print(f"⚠️ Warning: No data found for {seq_name} in CSV.")
        return

    for frame_idx, img_path in enumerate(tqdm(image_paths, leave=False)):
        img_cv = cv2.imread(img_path)
        if img_cv is None: continue

        frame_filename = os.path.basename(img_path)
        current_frame_dets = seq_data[seq_data['Filename'] == frame_filename].copy()

        # SORT 입력을 위한 데이터 정제
        if 'conf' not in current_frame_dets.columns:
            current_frame_dets['conf'] = 1.0
        dets_to_sort = current_frame_dets[['x1', 'y1', 'x2', 'y2', 'conf']].values

        # 1. 트래킹 업데이트
        track_dets = tracker.update(dets_to_sort)

        if len(track_dets) > 0:
            for trk in track_dets:
                x1, y1, x2, y2, track_id = trk

                # 2. CSV 원본 데이터와 IoU 매칭 (Location_ID, Action 정보 획득)
                matched_iou_val = -1
                matched_row = None
                for _, det_row in current_frame_dets.iterrows():
                    iou_val = iou(trk[:4], det_row[['x1', 'y1', 'x2', 'y2']].values)
                    if iou_val > matched_iou_val and iou_val > 0.1:
                        matched_iou_val = iou_val
                        matched_row = det_row

                if matched_row is not None:
                    loc_idx = int(matched_row['Location_ID'])
                    is_active = matched_row[['Brake', 'Left', 'Right', 'Hazard']].values.astype(int)

                    # 3. 위험도 계산
                    risk_result = risk_manager.calculate_risk(
                        track_id=int(track_id),
                        bbox=[x1, y1, x2, y2],
                        location=loc_idx,
                        actions=is_active,
                        fps=fps, img_w=w, img_h=h
                    )

                    # 4. 시각화
                    level = risk_result["level"]
                    color = RISK_COLORS.get(level, (0, 255, 0))
                    cv2.rectangle(img_cv, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                    loc_names = ["My", "Next", "Opposite", "RoadSide", "None"]
                    info_text = f"ID:{int(track_id)} | {loc_names[loc_idx] if loc_idx < 5 else '??'}"
                    cv2.putText(img_cv, info_text, (int(x1), int(y1) - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.putText(img_cv, f"{level} ({risk_result['score']})", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                    if level != "SAFE":
                        cv2.putText(img_cv, risk_result["reason"], (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(img_cv)
    out.release()

"""
# 3. 실행 부분
csv_sequence_names = all_det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0]).unique()
print(f"✅ CSV에 포함된 시퀀스 목록: {csv_sequence_names}")

for seq_name, img_list in sequences.items():
    processed_seq_name = seq_name.replace('.', '_')

    if processed_seq_name in csv_sequence_names:
        process_sequence_from_csv(
            processed_seq_name,
            img_list,
            OUTPUT_ROOT_RISK,
            all_det_df
        )
    else:
        print(f"⏩ Skipping {processed_seq_name}: No matching data found in CSV.")

print(f"\n✨ 위험도 분석 완료! 결과 경로: {OUTPUT_ROOT_RISK}")
"""
# --- 3. 실행 부분 (수정 버전) ---

# CSV 내의 시퀀스 이름 리스트 추출
csv_sequence_names = all_det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0]).unique()
print(f"✅ CSV에 포함된 시퀀스 목록: {csv_sequence_names}")

for seq_name, img_list in sequences.items():
    # 억지로 이름을 변환하지 않고,
    # 1. 원본 이름(seq_name)이 있는지 확인
    # 2. 혹은 마침표를 변환한 이름이 있는지 확인
    processed_seq_name = seq_name.replace('.', '_')

    target_name = None
    if seq_name in csv_sequence_names:
        target_name = seq_name
    elif processed_seq_name in csv_sequence_names:
        target_name = processed_seq_name

    if target_name:
        process_sequence_from_csv(
            target_name,
            img_list,
            OUTPUT_ROOT_RISK,
            all_det_df
        )
    else:
        print(f"⏩ Skipping {seq_name}: No matching data found in CSV.")

print(f"\n✨ 위험도 분석 완료! 결과 경로: {OUTPUT_ROOT_RISK}")

✅ CSV에 포함된 시퀀스 목록: ['bb_1_211026_vehicle_144_301.mp4']
⏩ Skipping bb_1_210701_vehicle_217_21650.mp4: No matching data found in CSV.
⏩ Skipping bb_1_160612_vehicle_212_53256.mp4: No matching data found in CSV.
⏩ Skipping bb_1_141024_vehicle_46_180.mp4: No matching data found in CSV.
🎬 Processing bb_1_211026_vehicle_144_301_mp4 from CSV...


⏩ Skipping bb_1_170304_vehicle_114_075.mp4: No matching data found in CSV.
⏩ Skipping bb_1_150402_vehicle_142_073.mp4: No matching data found in CSV.
⏩ Skipping bb_1_181121_vehicle_117_045.mp4: No matching data found in CSV.
⏩ Skipping bb_1_200714_vehicle_200_28840.mp4: No matching data found in CSV.
⏩ Skipping bb_1_210414_vehicle_244_50609.mp4: No matching data found in CSV.
⏩ Skipping bb_1_210114_vehicle_253_50073.mp4: No matching data found in CSV.
⏩ Skipping bb_1_210522_vehicle_315_061.mp4: No matching data found in CSV.

✨ 위험도 분석 완료! 결과 경로: /content/sago_SafeDrive_T4_HighPerf/risk_video


특정 파일만 저장하도록 설정 (이미지)

In [77]:
import pandas as pd
import cv2
import os
import numpy as np
from tqdm import tqdm

# Sort 트래커에서 iou 함수를 가져옵니다.
import importlib
import sort_tracker
importlib.reload(sort_tracker) # Reload the module to ensure latest changes are applied
from sort_tracker import Sort, iou

# 1. 먼저 CSV 파일을 로드합니다. (경로는 본인의 환경에 맞게 수정)
DET_CSV_PATH = "/content/SafeDrive_T4_HighPerf2/test_sequences_combined/Yeonhwi/Yeonhwi_combined.csv"
all_det_df = pd.read_csv(DET_CSV_PATH)

# OUTPUT_ROOT_RISK 변수 정의 추가
OUTPUT_ROOT_RISK = "/content/SafeDrive_T4_HighPerf2/integrated_risk_images"

# RISK_COLORS 정의 추가 (이전 셀에서 정의되었으나 명확성을 위해 추가)
RISK_COLORS = {
    "SAFE": (0, 255, 0),       # Green
    "WARNING": (0, 165, 255),  # Orange
    "DANGER": (0, 0, 255),
    "CAUTION": (0, 255, 255) # Yellow
}

import pandas as pd
import cv2
import os
import numpy as np
from tqdm import tqdm

# --- 수정 포인트: VideoWriter 대신 폴더 생성 및 imwrite 사용 ---

def process_sequence_from_csv_to_images(seq_name, image_paths, output_root, det_df):
    # 시퀀스별로 하위 폴더 생성 (예: .../integrated_risk_videos2/CounterClock2_Fog/)
    seq_output_dir = os.path.join(output_root, seq_name)
    os.makedirs(seq_output_dir, exist_ok=True)

    image_paths = sorted(image_paths)
    if not image_paths: return

    # Init Tracker & Risk Manager
    tracker = Sort(max_age=5, min_hits=1, iou_threshold=0.3)
    risk_manager = IntentAwareRiskManager(window_size=10)

    print(f"📸 Processing {seq_name} (Saving as Images)...")

    # 데이터 필터링 로직 (기존과 동일)
    seq_data = det_df[det_df['Filename'].apply(lambda x: x.startswith(seq_name))].copy()

    for frame_idx, img_path in enumerate(tqdm(image_paths, leave=False)):
        img_cv = cv2.imread(img_path)
        if img_cv is None: continue
        h, w = img_cv.shape[:2]

        # CSV 데이터 추출
        frame_filename = os.path.basename(img_path)
        current_frame_dets = seq_data[seq_data['Filename'] == frame_filename].copy()

        if 'conf' not in current_frame_dets.columns:
            current_frame_dets['conf'] = 1.0
        dets_to_sort = current_frame_dets[['x1', 'y1', 'x2', 'y2', 'conf']].values

        # SORT Tracking
        track_dets = tracker.update(dets_to_sort)

        # 위험도 계산 및 드로잉
        if len(track_dets) > 0:
            for trk in track_dets:
                x1, y1, x2, y2, track_id = trk

                # IoU 매칭 (기존 로직 유지)
                matched_iou_val = -1
                matched_row = None
                for _, det_row in current_frame_dets.iterrows():
                    iou_val = iou(trk[:4], det_row[['x1', 'y1', 'x2', 'y2']].values)
                    if iou_val > matched_iou_val and iou_val > 0.1:
                        matched_iou_val = iou_val
                        matched_row = det_row

                if matched_row is not None:
                    loc_idx = int(matched_row['Location_ID'])
                    is_active = matched_row[['Brake', 'Left', 'Right', 'Hazard']].values.astype(int)

                    risk_result = risk_manager.calculate_risk(
                        track_id=int(track_id),
                        bbox=[x1, y1, x2, y2],
                        location=loc_idx,
                        actions=is_active,
                        fps=10.0, img_w=w, img_h=h
                    )

                    # 시각화 부분
                    level = risk_result["level"]
                    color = RISK_COLORS.get(level, (0, 255, 0))
                    cv2.rectangle(img_cv, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                    loc_names = ["My", "Next", "Opposite", "RoadSide", "None"]
                    info_text = f"ID:{int(track_id)} | {loc_names[loc_idx] if loc_idx < 5 else '??'}"
                    cv2.putText(img_cv, info_text, (int(x1), int(y1) - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.putText(img_cv, f"{level} ({risk_result['score']})", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    if level != "SAFE":
                        cv2.putText(img_cv, risk_result["reason"], (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # --- [변경 파트: 영상 저장 대신 이미지 저장] ---
        # 원본 파일명 앞에 'risk_'를 붙여 저장하거나 프레임 번호로 저장
        save_filename = f"risk_{frame_filename}"
        save_path = os.path.join(seq_output_dir, save_filename)
        cv2.imwrite(save_path, img_cv)

"""
# --- 실행 부분 ---
for seq_name, img_list in sequences.items():
    process_sequence_from_csv_to_images(
        seq_name,
        img_list,
        OUTPUT_ROOT_RISK,
        all_det_df
    )
"""

# --- 실행 부분 ---

# 1. CSV 파일 안에 어떤 시퀀스 이름들이 있는지 리스트로 추출합니다.
# Filename 컬럼의 값(예: 'CounterClock2_Fog_001.png')에서 시퀀스 이름만 뽑아냅니다.
csv_sequence_names = all_det_df['Filename'].apply(lambda x: x.rsplit('_', 1)[0]).unique()

print(f"✅ CSV에 포함된 시퀀스 목록: {csv_sequence_names}")

for seq_name, img_list in sequences.items():
    # 2. 현재 처리하려는 폴더명(seq_name)이 CSV 리스트에 들어있는지 확인합니다.
    if seq_name in csv_sequence_names:
        process_sequence_from_csv_to_images(
            seq_name,
            img_list,
            OUTPUT_ROOT_RISK,
            all_det_df
        )
    else:
        # CSV에 정보가 없는 폴더는 그냥 건너뜁니다.
        print(f"⏩ Skipping {seq_name}: No matching data found in CSV.")

    #out.write(img_cv)

   # out.release()

✅ CSV에 포함된 시퀀스 목록: ['Yeonhwi']
📸 Processing Yeonhwi (Saving as Images)...


⏩ Skipping CounterClock2_Fog: No matching data found in CSV.
⏩ Skipping ETRINear_Fog: No matching data found in CSV.
⏩ Skipping OisamtoBanseok_Snow: No matching data found in CSV.
⏩ Skipping RAIN_Noeun: No matching data found in CSV.
⏩ Skipping TailLight27: No matching data found in CSV.
⏩ Skipping TailLight57: No matching data found in CSV.
⏩ Skipping TailLight87: No matching data found in CSV.
⏩ Skipping TailLight45: No matching data found in CSV.
⏩ Skipping TailLight04: No matching data found in CSV.


영상 파일 드라이브로 복사

In [112]:
# -r (recursive): 하위 폴더 포함 전체 복사
!cp -r "/content/sago_SafeDrive_T4_HighPerf/risk_video" "/content/drive/MyDrive/OD_project/sago"

이미지 파일 드라이브로 복사

In [80]:
# -r (recursive): 하위 폴더 포함 전체 복사
!cp -r "/content/SafeDrive_T4_HighPerf2/integrated_risk_images" "/content/drive/MyDrive/OD_project/risk_results_images2/"